<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FLIGHT_PLANNING_BoltzmannMachine_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade ipykernel -q
!pip install --upgrade IPython -q
!pip install datasets -q
!pip install gym -q
!pip install torchviz -q
!pip install torch-tb-profiler -q
!pip install tensorboard -q
!pip install datasets -q
!pip install --upgrade hyperopt -q
!pip install --upgrade setuptools -q
!pip install kagglehub --upgrade -q

https://www.kaggle.com/datasets/open-flights/airports-train-stations-and-ferry-terminals?select=airports-extended.csv



In [4]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("open-flights/flight-route-database")
print("Path to dataset files:", path)

# Load routes data
routes_data = pd.read_csv(path + "/routes.csv")


import kagglehub

# Download latest version
path = kagglehub.dataset_download("open-flights/airports-train-stations-and-ferry-terminals")

print("Path to dataset files:", path)

# Load airports data - Changed from "routes.csv" to "airports.csv"
# The dataset 'open-flights/airports-train-stations-and-ferry-terminals' was likely intended
# and it contains a single file called 'airports-extended.csv', so let's try that.
airports_data = pd.read_csv(path + "/airports-extended.csv")

Path to dataset files: /root/.cache/kagglehub/datasets/open-flights/flight-route-database/versions/1


100%|██████████| 437k/437k [00:00<00:00, 597kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/open-flights/airports-train-stations-and-ferry-terminals/versions/1


In [5]:
airports_data.head()

,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081689834590001,145.391998291,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
0,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
2,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
3,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
4,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports


In [6]:
routes_data

,airline,airline ID,source airport,source airport id,destination apirport,destination airport id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [9]:
import gym
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import pandas as pd
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
import sys
import logging
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import io  # To create DataFrame from your provided data

# Suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pkg_resources")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="hyperopt")
logging.captureWarnings(True)
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Check for GPU and set device
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device('cpu')
    print("Using CPU")

# Boltzmann Machine Class
class BoltzmannMachine(nn.Module):
    def __init__(self, num_visible, num_hidden):
        super(BoltzmannMachine, self).__init__()
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = nn.Parameter(torch.randn(num_visible, num_hidden) * 0.1)
        self.hidden_bias = nn.Parameter(torch.zeros(num_hidden))
        self.visible_bias = nn.Parameter(torch.zeros(num_visible))
        self.to(device)

    def sample_hidden(self, visible):
        hidden_activations = torch.matmul(visible, self.weights) + self.hidden_bias
        hidden_probs = torch.sigmoid(hidden_activations)
        hidden_states = torch.bernoulli(hidden_probs)
        return hidden_states

    def sample_visible(self, hidden):
        visible_activations = torch.matmul(hidden, self.weights.t()) + self.visible_bias
        visible_probs = torch.sigmoid(visible_activations)
        visible_states = torch.bernoulli(visible_probs)
        return visible_states

    def train(self, data, epochs=10, learning_rate=0.1):
        data = torch.tensor(data, dtype=torch.float32, device=self.weights.device)
        for epoch in range(epochs):
            pos_hidden_probs = torch.sigmoid(torch.matmul(data, self.weights) + self.hidden_bias)
            pos_hidden_states = torch.bernoulli(pos_hidden_probs)
            pos_associations = torch.matmul(data.t(), pos_hidden_probs)

            neg_visible_states = self.sample_visible(pos_hidden_states)
            neg_hidden_probs = torch.sigmoid(torch.matmul(neg_visible_states, self.weights) + self.hidden_bias)
            neg_associations = torch.matmul(neg_visible_states.t(), neg_hidden_probs)

            self.weights.data += learning_rate * (pos_associations - neg_associations) / data.shape[0]
            self.hidden_bias.data += learning_rate * (torch.sum(pos_hidden_probs, dim=0) - torch.sum(neg_hidden_probs, dim=0)) / data.shape[0]
            self.visible_bias.data += learning_rate * (torch.sum(data, dim=0) - torch.sum(neg_visible_states, dim=0)) / data.shape[0]

# Flight Stops Prediction Environment Class
import gym
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from gym.spaces import Discrete, Dict, Box

class FlightStopsEnv(gym.Env):
    def __init__(self, routes_data):
        self.routes_data = routes_data
        self.action_space = Discrete(3)  # 0: 0 stops, 1: 1 stop, 2: 2 or more stops
        self.feature_cols = ['airline ID', 'source airport id', 'destination airport id', 'codeshare', 'equipment']

        # Preprocessing
        categorical_features = self.feature_cols  # All features are categorical
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),
            ],
            remainder='passthrough'  # No numerical features to passthrough
        )

        # Fit the preprocessor and get the number of features after preprocessing
        self.preprocessor.fit(pd.DataFrame(self.routes_data[self.feature_cols]))
        self.num_features = self.preprocessor.transform(pd.DataFrame(self.routes_data[self.feature_cols][:1])).shape[1]

        # Define observation space with the correct shape
        self.observation_space = Dict({
            'features': Box(low=0, high=1, shape=(self.num_features,), dtype=np.float32)
        })

        self.current_route_index = 0
        self.reset()

    def reset(self):
        self.current_route_index = (self.current_route_index + 1) % len(self.routes_data)
        self.features = self.routes_data.iloc[self.current_route_index][self.feature_cols].values
        self.features = self.preprocess_features(self.features)  # Preprocess features here
        return self._get_obs()

    def step(self, action):
        actual_stops = self.routes_data.iloc[self.current_route_index]['stops']
        actual_outcome = 0 if actual_stops == 0 else (1 if actual_stops == 1 else 2)
        reward = self.reward_function(action, actual_outcome)
        done = True
        return self._get_obs(), reward, done, {}

    def reward_function(self, action, actual_outcome):
        return 1 if action == actual_outcome else -1

    def _get_obs(self):
        return {'features': self.features}

    def preprocess_features(self, features):
        if features.ndim == 1:
            features = features.reshape(1, -1)
        features_df = pd.DataFrame(features, columns=self.feature_cols)
        features_transformed = self.preprocessor.transform(features_df)
        return features_transformed

# Q-Learning Agent Class
class QLearningAgent:
    def __init__(self, observation_space, action_space, learning_rate=0.01, discount_factor=0.9, epsilon=0.2,
                 epsilon_decay=0.995, device=None):
        self.observation_space = observation_space
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.device = device
        self.q_table = {}
        self.returns = {}

    def get_action(self, state):
        features = state['features'].reshape(1, -1)
        try:
            state_tuple = tuple(features.flatten())
        except TypeError:  # Handle cases where conversion to tuple fails
            print("Error: Could not convert features to tuple. Features:", features)
            return self.action_space.sample()  # Return a random action

        if state_tuple not in self.q_table:
            self.q_table[state_tuple] = np.zeros(self.action_space.n, dtype=np.float32)
            self.q_table[state_tuple][:] = 0.1

        if np.random.rand() < self.epsilon:
            return self.action_space.sample()
        else:
            q_values = self.q_table[state_tuple]
            action = np.argmax(q_values)
            return action

    def update(self, episode):
        states, actions, rewards = zip(*episode)
        G = 0
        for t in reversed(range(len(states))):
            G = self.discount_factor * G + rewards[t]
            state_tuple = tuple(states[t]['features'].flatten())
            state_action = (state_tuple, actions[t])

            if state_action not in self.returns:
                self.returns[state_action] = []

            self.returns[state_action].append(G)
            self.q_table[state_tuple][actions[t]] = np.mean(self.returns[state_action])

        self.epsilon = max(0.01, self.epsilon * self.epsilon_decay)


# Data
data_string = """airline,airline ID,source airport,source airport id,destination apirport,destination airport id,codeshare,stops,equipment
2B,410,AER,2965,KZN,2990,NaN,0,CR2
2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2B,410,ASF,2966,MRV,2962,NaN,0,CR2
2B,410,CEK,2968,KZN,2990,NaN,0,CR2
2B,410,CEK,2968,OVB,4078,NaN,0,CR2"""

routes_data = pd.read_csv(io.StringIO(data_string))

# Fill NaN values in 'codeshare'
routes_data['codeshare'] = routes_data['codeshare'].fillna('N')

# Create environment
env = FlightStopsEnv(routes_data)

# Generate training data for the Boltzmann machine
training_data = routes_data[env.feature_cols].values
training_data = env.preprocess_features(training_data)
training_data = torch.tensor(training_data, dtype=torch.float32, device=device)

# Initialize the Boltzmann machine
num_visible = env.num_features  # Use num_features from the environment
bm = BoltzmannMachine(num_visible=num_visible, num_hidden=50).to(device)


# Define the objective function for hyperopt
def objective(params):
    agent = QLearningAgent(env.observation_space, env.action_space,
                           learning_rate=params['learning_rate'],
                           discount_factor=params['discount_factor'],
                           epsilon=params['epsilon'],
                           epsilon_decay=params['epsilon_decay'],
                           device=device)

    for episode in range(500):
        state = env.reset()
        done = False
        episode_data = []
        while not done:
            bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device).reshape(1, -1)
            hidden_state = bm.sample_hidden(bm_input)

            processed_state = {
                'features': state['features'],
                'hidden_state': hidden_state.cpu().detach().numpy().flatten()
            }

            processed_state_for_action = {'features': processed_state['features']}

            action = agent.get_action(processed_state_for_action)
            next_state, reward, done, _ = env.step(action)
            episode_data.append((processed_state_for_action, action, reward))
            state = next_state
        agent.update(episode_data)

    # Evaluation for hyperopt
    total_eval_reward = 0
    for _ in range(100):
        state = env.reset()
        done = False
        while not done:
            bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device).reshape(1, -1)
            hidden_state = bm.sample_hidden(bm_input)
            processed_state = {
                'features': state['features'],
                'hidden_state': hidden_state.cpu().detach().numpy().flatten()
            }
            processed_state_for_action = {'features': processed_state['features']}
            action = agent.get_action(processed_state_for_action)
            next_state, reward, done, _ = env.step(action)
            total_eval_reward += reward
            state = next_state

    avg_eval_reward = total_eval_reward / 100
    return {'loss': -avg_eval_reward, 'status': STATUS_OK}

# Define the hyperparameter search space
space = {
    'discount_factor': hp.uniform('discount_factor', 0.7, 0.99),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    'epsilon': hp.uniform('epsilon', 0.1, 0.3),
    'epsilon_decay': hp.uniform('epsilon_decay', 0.99, 0.999)
}

# Perform hyperparameter optimization
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

print(f"Best hyperparameters: {best}")

# Create the best agent
best_agent = QLearningAgent(env.observation_space, env.action_space,
                           discount_factor=best['discount_factor'],
                           learning_rate=best['learning_rate'],
                           epsilon=best['epsilon'],
                           epsilon_decay=best['epsilon_decay'],
                           device=device)

# Final Training
for episode in tqdm(range(2000), desc="Final Training Progress"):
    state = env.reset()
    done = False
    episode_data = []
    while not done:
        bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device).reshape(1, -1)
        hidden_state = bm.sample_hidden(bm_input)

        processed_state = {
            'features': state['features'],
            'hidden_state': hidden_state.cpu().detach().numpy().flatten()
        }

        processed_state_for_action = {'features': processed_state['features']}

        action = best_agent.get_action(processed_state_for_action)
        next_state, reward, done, _ = env.step(action)
        episode_data.append((processed_state_for_action, action, reward))
        state = next_state
    best_agent.update(episode_data)
    if episode % 50 == 0:
        bm.train(training_data, epochs=10, learning_rate=0.1)

# Evaluation Phase
total_eval_reward = 0
correct_predictions = 0
for episode in tqdm(range(100), desc="Evaluation Progress"):
    state = env.reset()
    done = False
    while not done:
        bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device).reshape(1, -1)
        hidden_state = bm.sample_hidden(bm_input)

        processed_state = {
            'features': state['features'],
            'hidden_state': hidden_state.cpu().detach().numpy().flatten()
        }

        processed_state_for_action = {'features': processed_state['features']}

        action = best_agent.get_action(processed_state_for_action)
        next_state, reward, done, _ = env.step(action)
        total_eval_reward += reward

        actual_stops = env.routes_data.iloc[env.current_route_index]['stops']
        actual_outcome = 0 if actual_stops == 0 else (1 if actual_stops == 1 else 2)
        if action == actual_outcome:
            correct_predictions += 1
        state = next_state

avg_eval_reward = total_eval_reward / 100
accuracy = correct_predictions / 100
print(f"\nAverage evaluation reward: {avg_eval_reward}")
print(f"Accuracy: {accuracy}")

Using GPU: NVIDIA L4
100%|██████████| 50/50 [01:33<00:00,  1.88s/trial, best loss: -1.0]
Best hyperparameters: {'discount_factor': 0.9890478919576083, 'epsilon': 0.23496129791751091, 'epsilon_decay': 0.9933579115825788, 'learning_rate': 0.09019571603658609}


Evaluation Progress: 100%|██████████| 100/100 [00:00<00:00, 312.85it/s]


Average evaluation reward: 1.0
Accuracy: 1.0


## Deep Q-Networks (DQN)

In [11]:
!pip install gym -q
!pip install --upgrade hyperopt -q

import gym
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import pandas as pd
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
import sys
import logging
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import io  # To create DataFrame from your provided data
import random
from collections import deque

# Suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pkg_resources")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="hyperopt")
logging.captureWarnings(True)
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Check for GPU and set device
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device('cpu')
    print("Using CPU")

# Boltzmann Machine Class
class BoltzmannMachine(nn.Module):
    def __init__(self, num_visible, num_hidden):
        super(BoltzmannMachine, self).__init__()
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = nn.Parameter(torch.randn(num_visible, num_hidden) * 0.1)
        self.hidden_bias = nn.Parameter(torch.zeros(num_hidden))
        self.visible_bias = nn.Parameter(torch.zeros(num_visible))
        self.to(device)

    def sample_hidden(self, visible):
        hidden_activations = torch.matmul(visible, self.weights) + self.hidden_bias
        hidden_probs = torch.sigmoid(hidden_activations)
        hidden_states = torch.bernoulli(hidden_probs)
        return hidden_states

    def sample_visible(self, hidden):
        visible_activations = torch.matmul(hidden, self.weights.t()) + self.visible_bias
        visible_probs = torch.sigmoid(visible_activations)
        visible_states = torch.bernoulli(visible_probs)
        return visible_states

    def train(self, data, epochs=10, learning_rate=0.1):
        data = torch.tensor(data, dtype=torch.float32, device=self.weights.device)
        for epoch in range(epochs):
            pos_hidden_probs = torch.sigmoid(torch.matmul(data, self.weights) + self.hidden_bias)
            pos_hidden_states = torch.bernoulli(pos_hidden_probs)
            pos_associations = torch.matmul(data.t(), pos_hidden_probs)

            neg_visible_states = self.sample_visible(pos_hidden_states)
            neg_hidden_probs = torch.sigmoid(torch.matmul(neg_visible_states, self.weights) + self.hidden_bias)
            neg_associations = torch.matmul(neg_visible_states.t(), neg_hidden_probs)

            self.weights.data += learning_rate * (pos_associations - neg_associations) / data.shape[0]
            self.hidden_bias.data += learning_rate * (torch.sum(pos_hidden_probs, dim=0) - torch.sum(neg_hidden_probs, dim=0)) / data.shape[0]
            self.visible_bias.data += learning_rate * (torch.sum(data, dim=0) - torch.sum(neg_visible_states, dim=0)) / data.shape[0]

# Flight Stops Prediction Environment Class
class FlightStopsEnv(gym.Env):
    def __init__(self, routes_data):
        self.routes_data = routes_data
        self.action_space = gym.spaces.Discrete(3)  # 0: 0 stops, 1: 1 stop, 2: 2 or more stops
        self.feature_cols = ['airline ID', 'source airport id', 'destination airport id', 'codeshare', 'equipment']

        # Preprocessing
        categorical_features = self.feature_cols  # All features are categorical
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),
            ],
            remainder='passthrough'  # No numerical features to passthrough
        )

        # Fit the preprocessor and get the number of features after preprocessing
        self.preprocessor.fit(pd.DataFrame(self.routes_data[self.feature_cols]))
        self.num_features = self.preprocessor.transform(pd.DataFrame(self.routes_data[self.feature_cols][:1])).shape[1]

        # Define observation space with the correct shape
        self.observation_space = gym.spaces.Dict({
            'features': gym.spaces.Box(low=0, high=1, shape=(self.num_features,), dtype=np.float32)
        })

        self.current_route_index = 0
        self.reset()

    def reset(self):
        self.current_route_index = (self.current_route_index + 1) % len(self.routes_data)
        self.features = self.routes_data.iloc[self.current_route_index][self.feature_cols].values
        self.features = self.preprocess_features(self.features)  # Preprocess features here
        return self._get_obs()

    def step(self, action):
        actual_stops = self.routes_data.iloc[self.current_route_index]['stops']
        actual_outcome = 0 if actual_stops == 0 else (1 if actual_stops == 1 else 2)
        reward = self.reward_function(action, actual_outcome)
        done = True
        return self._get_obs(), reward, done, {}

    def reward_function(self, action, actual_outcome):
        return 1 if action == actual_outcome else -1

    def _get_obs(self):
        return {'features': self.features}

    def preprocess_features(self, features):
        if features.ndim == 1:
            features = features.reshape(1, -1)
        features_df = pd.DataFrame(features, columns=self.feature_cols)
        features_transformed = self.preprocessor.transform(features_df)
        return features_transformed

# DQN Agent Class
class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, batch_size=64, memory_size=10000, device=None): # Add device as a parameter
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        self.memory = deque(maxlen=memory_size)
        self.model = self._build_model()
        self.target_model = self._build_model()  # Target network
        self.target_model.load_state_dict(self.model.state_dict())  # Initialize target network with the same weights as the main network
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.loss_fn = nn.MSELoss()
        self.device = device # Assign the device attribute
        self.model.to(self.device) # Move the model to the specified device
        self.target_model.to(self.device) # Move the target model to the specified device

    def _build_model(self):
        model = nn.Sequential(
            nn.Linear(self.state_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, self.action_size)
        )
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            state_tensor = torch.tensor(state['features'], dtype=torch.float32, device=self.device).unsqueeze(0)
            act_values = self.model(state_tensor)
            return torch.argmax(act_values).item()

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*minibatch)

        # Reshape features within states and next_states
        states = [{'features': s['features'].reshape(1, -1)} for s in states]
        next_states = [{'features': s['features'].reshape(1, -1)} for s in next_states]

        states_tensor = torch.tensor(np.array([s['features'] for s in states]), dtype=torch.float32, device=self.device).squeeze(1)
        actions_tensor = torch.tensor(actions, dtype=torch.long, device=self.device).unsqueeze(1)
        rewards_tensor = torch.tensor(rewards, dtype=torch.float32, device=self.device)
        next_states_tensor = torch.tensor(np.array([s['features'] for s in next_states]), dtype=torch.float32, device=self.device).squeeze(1)
        dones_tensor = torch.tensor(dones, dtype=torch.bool, device=self.device)

        q_values = self.model(states_tensor).gather(1, actions_tensor)
        next_q_values = self.target_model(next_states_tensor).max(1)[0].detach()
        target_q_values = rewards_tensor + self.gamma * next_q_values * (~dones_tensor)

        loss = self.loss_fn(q_values, target_q_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update epsilon
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

# Data
data_string = """airline,airline ID,source airport,source airport id,destination apirport,destination airport id,codeshare,stops,equipment
2B,410,AER,2965,KZN,2990,NaN,0,CR2
2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2B,410,ASF,2966,MRV,2962,NaN,0,CR2
2B,410,CEK,2968,KZN,2990,NaN,0,CR2
2B,410,CEK,2968,OVB,4078,NaN,0,CR2"""

routes_data = pd.read_csv(io.StringIO(data_string))
routes_data['codeshare'] = routes_data['codeshare'].fillna('N')

# Create environment
env = FlightStopsEnv(routes_data)

# Generate training data for the Boltzmann machine
training_data = routes_data[env.feature_cols].values
training_data = env.preprocess_features(training_data)
training_data = torch.tensor(training_data, dtype=torch.float32, device=device)

# Initialize the Boltzmann machine
num_visible = env.num_features
bm = BoltzmannMachine(num_visible=num_visible, num_hidden=50).to(device)

# Initialize the DQN agent
state_size = env.observation_space['features'].shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size, device=device)

# Training loop
for episode in tqdm(range(2000), desc="Training Progress"):
    state = env.reset()
    done = False
    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        agent.replay()  # Train the agent
    if episode % 50 == 0:  # Update target network every 50 episodes
        agent.update_target_model()
        bm.train(training_data, epochs=10, learning_rate=0.1)

# Evaluation Phase
total_eval_reward = 0
correct_predictions = 0
for episode in tqdm(range(100), desc="Evaluation Progress"):
    state = env.reset()
    done = False
    while not done:
        action = agent.act(state)  # Get action from the DQN agent
        next_state, reward, done, _ = env.step(action)
        total_eval_reward += reward
        actual_stops = env.routes_data.iloc[env.current_route_index]['stops']
        actual_outcome = 0 if actual_stops == 0 else (1 if actual_stops == 1 else 2)
        if action == actual_outcome:
            correct_predictions += 1
        state = next_state

avg_eval_reward = total_eval_reward / 100
accuracy = correct_predictions / 100
print(f"\nAverage evaluation reward: {avg_eval_reward}")
print(f"Accuracy: {accuracy}")

Using GPU: NVIDIA L4


Evaluation Progress: 100%|██████████| 100/100 [00:00<00:00, 299.25it/s]


Average evaluation reward: 1.0
Accuracy: 1.0
